# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [90]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [91]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [92]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Maximizing Water Flow

**Parameters:**

* **T:** Number of time steps (T = 10)
* **N:** Set of nodes {A, B, C, D, E, F, G}
* **E:** Set of edges representing pipes {(A,B), (A,C), (A,F), (B,C), (B,D), (B,E), (C,E), (C,F), (D,E), (E,G), (F,B), (F,G)}
* **c<sub>ij</sub>:** Capacity of pipe (i,j) ∈ E, as given in the problem statement.
* **f(t):** Inflow function at point A, f(t) = max(-4/30t<sup>3</sup>+t<sup>2</sup>-0.234t+3, 0)

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow of water through pipe (i,j) ∈ E at time t = 1,...,T
* **y<sub>ij</sub>:** Binary variable indicating whether pipe (i,j) ∈ E is activated (y<sub>ij</sub> = 1) or deactivated (y<sub>ij</sub> = 0)
* **r<sub>it</sub>:** Amount of water released from node i ∈ N at time t = 1,...,T

**Objective Function:**

Maximize the total water flow out of point G over all time steps:

```
Maximize  ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow Conservation:** For each node i ∈ N and time t = 1,...,T, the total inflow must equal the total outflow plus any released water:
    * At point A: f(t) = ∑<sub>j:(A,j)∈E</sub> x<sub>Ajt</sub> 
    * At other points: ∑<sub>j:(j,i)∈E</sub> x<sub>jit</sub> = ∑<sub>j:(i,j)∈E</sub> x<sub>ijt</sub> + r<sub>it</sub>
2. **Pipe Capacity:** Flow through each pipe cannot exceed its capacity:
    * x<sub>ijt</sub> ≤ c<sub>ij</sub> * y<sub>ij</sub> for all (i,j) ∈ E and t = 1,...,T
3. **Pipe Activation Limit:** Only 10 pipes can be activated:
    * ∑<sub>(i,j)∈E</sub> y<sub>ij</sub> ≤ 10
4. **Non-negativity:** All variables must be non-negative:
    * x<sub>ijt</sub> ≥ 0 for all (i,j) ∈ E and t = 1,...,T
    * r<sub>it</sub> ≥ 0 for all i ∈ N and t = 1,...,T
    * y<sub>ij</sub> ∈ {0,1} for all (i,j) ∈ E

This model represents a mixed-integer linear programming problem, which can be solved using appropriate optimization software to find the optimal flow configuration that maximizes water output at point G. 


## 3. Generate the pyomo code

In [93]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [94]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5, ('A', 'C'): 3, ('A', 'F'): 4, ('B', 'C'): 2,
    ('B', 'D'): 1, ('B', 'E'): 6, ('C', 'E'): 4, ('C', 'F'): 3,
    ('D', 'E'): 2, ('E', 'G'): 7, ('F', 'B'): 2, ('F', 'G'): 5
}

# Parameters
T = 10  # Number of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
E = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'),
     ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of edges

# Create model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(E, range(1, T + 1), within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, range(1, T + 1), within=pyo.NonNegativeReals)  # Water released from nodes

# Objective function: Maximize outflow at G
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in range(1, T + 1)), sense=pyo.maximize)

# Constraints
# Flow conservation
model.flow_conservation = pyo.ConstraintList()
for t in range(1, T + 1):
    model.flow_conservation.add(
        expr=max(-4 / 30 * t ** 3 + t ** 2 - 0.234 * t + 3, 0) == sum(model.x[i, j, t] for (i, j) in E if i == 'A'))
    for i in N - {'A'}:
        model.flow_conservation.add(
            expr=sum(model.x[j, i, t] for (j, i) in E if i == j) ==
                 sum(model.x[i, j, t] for (i, j) in E if i == j) + model.r[i, t])

# Pipe capacity
model.pipe_capacity = pyo.ConstraintList()
for (i, j) in E:
    for t in range(1, T + 1):
        model.pipe_capacity.add(expr=model.x[i, j, t] <= c[i, j] * model.y[i, j])

# Pipe activation limit
model.activation_limit = pyo.Constraint(expr=sum(model.y[i, j] for (i, j) in E) <= 10)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'gurobi' or 'cplex'
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective value:", pyo.value(model.obj))
print("Pipe Activation:")
for (i, j) in E:
    if pyo.value(model.y[i, j]) == 1:
        print(f"Pipe ({i}, {j}) is activated")
# You can also access and print the flow values (model.x) and released water (model.r) as needed
```

This code first defines the sample data for pipe capacities in the dictionary `c`. You need to replace these values with your actual data. 

The rest of the code implements the model as described in the problem statement, including the objective function, constraints, and variable definitions. Finally, it uses the 'glpk' solver to solve the problem and then prints the results, including the objective value and which pipes are activated in the optimal solution.


## 4. Input problem data and try running the generated code

In [106]:
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}

# Parameters
T = 10  # Number of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
E = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'),
     ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of edges

# Create model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(E, range(1, T + 1), within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, range(1, T + 1), within=pyo.NonNegativeReals)  # Water released from nodes

# Objective function: Maximize outflow at G
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in range(1, T + 1)), sense=pyo.maximize)

# Constraints
# Flow conservation
model.flow_conservation = pyo.ConstraintList()
for t in range(1, T + 1):
    model.flow_conservation.add(
        expr=max(-4 / 30 * t ** 3 + t ** 2 - 0.234 * t + 3, 0) == sum(model.x[i, j, t] for (i, j) in E if i == 'A'))
    for i in N - {'A'}:
        model.flow_conservation.add(
            expr=sum(model.x[j, i, t] for (j, i) in E if i == j) ==
                 sum(model.x[i, j, t] for (i, j) in E if i == j) + model.r[i, t])

# Pipe capacity
model.pipe_capacity = pyo.ConstraintList()
for (i, j) in E:
    for t in range(1, T + 1):
        model.pipe_capacity.add(expr=model.x[i, j, t] <= c[i, j] * model.y[i, j])

# Pipe activation limit
model.activation_limit = pyo.Constraint(expr=sum(model.y[i, j] for (i, j) in E) <= 10)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'gurobi' or 'cplex'
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective value:", pyo.value(model.obj))
print("Pipe Activation:")
for (i, j) in E:
    if pyo.value(model.y[i, j]) == 1:
        print(f"Pipe ({i}, {j}) is activated")

Optimal Solution:
ERROR: evaluating object as numeric value: x[E,G,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[E,G,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[E,G,1]


ValueError: No value for uninitialized NumericValue object x[E,G,1]

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [107]:
print(response.text)

## Mathematical Optimization Model for Maximizing Water Flow

**Parameters:**

* **T:** Number of time steps (T = 10)
* **N:** Set of nodes {A, B, C, D, E, F, G}
* **E:** Set of edges representing pipes {(A,B), (A,C), (A,F), (B,C), (B,D), (B,E), (C,E), (C,F), (D,E), (E,G), (F,B), (F,G)}
* **c<sub>ij</sub>:** Capacity of pipe (i,j) ∈ E, as given in the problem statement.
* **f(t):** Inflow function at point A, f(t) = max(-4/30t<sup>3</sup>+t<sup>2</sup>-0.234t+3, 0)

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow of water through pipe (i,j) ∈ E at time t = 1,...,T
* **y<sub>ij</sub>:** Binary variable indicating whether pipe (i,j) ∈ E is activated (y<sub>ij</sub> = 1) or deactivated (y<sub>ij</sub> = 0)
* **r<sub>it</sub>:** Amount of water released from node i ∈ N at time t = 1,...,T

**Objective Function:**

Maximize the total water flow out of point G over all time steps:

```
Maximize  ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow Conservation:*

In [108]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5, ('A', 'C'): 3, ('A', 'F'): 4, ('B', 'C'): 2,
    ('B', 'D'): 1, ('B', 'E'): 6, ('C', 'E'): 4, ('C', 'F'): 3,
    ('D', 'E'): 2, ('E', 'G'): 7, ('F', 'B'): 2, ('F', 'G'): 5
}

# Parameters
T = 10  # Number of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
E = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'),
     ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of edges

# Create model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(E, range(1, T + 1), within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, range(1, T + 1), within=pyo.NonNegativeReals)  # Water released from nodes

# Objective function: Maximize outflow at G
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in range(1, T